In [1]:
import requests
from bs4 import BeautifulSoup 
from tqdm import tqdm
import pandas as pd

In [2]:
banks = ["akbank","halkbank","is%20bankasi","yapi%20kredi","vakifbank","garanti","denizbank"]
web_links = {
    "borsa_gundem":"https://www.borsagundem.com/ara/{bank}/{i}",
    "bloomberght":"https://www.bloomberght.com/liste/arama/{i}?ara={bank}&baslangic_tarihi=&bitis_tarihi=",
    "hurriyet":"http://bigpara.hurriyet.com.tr/arama/?firstDate=2015-06-01&secondDate=2020-09-28&includeKap=False&page={i}&s={bank}",
    "finans_gundem":"https://www.finansgundem.com/ara/{bank}/{i}"
    
}

In [3]:
def borsa_gundem(array,bank):
    for i in tqdm(range(1,75)):
        page = requests.get(web_links["borsa_gundem"].format(bank=bank,i=i))
        sp = BeautifulSoup(page.content,"html.parser")
        liste = sp.find("div",{"class":"srch"}).find_all("li")
        for l in liste: 
            try:
                title = l.strong.text
                tarih = l.small.text
                tarih = tarih[:tarih.find(",")]
                content = ""
                pg = requests.get(l.a.get("href"))
                psp = BeautifulSoup(pg.content,"html.parser")
                for i in list(psp.find("div",{"class":"dtail"}).find_all("p")[:-2]):
                    content += i.text.replace("\xa0"," ")
                if not content:
                    continue
                array.append([title,content,tarih,bank])
            except Exception as e:
                print(l.a.get("href"),"  ",e)
                continue    

def bloomberght(array,bank):
    adres = "https://www.bloomberght.com"
    for i in tqdm(range(1,50)):
        page = requests.get(web_links["bloomberght"].format(bank=bank,i=i))
        sp = BeautifulSoup(page.content,"html.parser")
        liste = [adres + i.a.get("href") for i in sp.find_all("div",{"class":"box box-12 item"}) if not "/video/" in i.a.get("href")]

        for l in liste:
            try:
                pg = requests.get(l)
                psp = BeautifulSoup(pg.content,"html.parser")

                title = psp.h1.span.text
                tarih = psp.find("div",{"class":"news-info"}).span.time.text
                tarih = " ".join(tarih.split()[:3])
                content =""
                for i in list(psp.article.find_all("p")):
                    content += i.text
                array.append([title,content,tarih,bank])
            except Exception as e:
                print(l, "  ", e)
                
def hurriyet(array,bank):
    adres = "http://bigpara.hurriyet.com.tr"
    for i in tqdm(range(1,30)):   
        page = requests.get(web_links["hurriyet"].format(bank=bank,i=i))
        sp = BeautifulSoup(page.content, "html.parser")
        liste = sp.find("div",{"class":"aramasonuc"}).find_all("li")
        
        for l in liste:
            try:
                title = l.a.text
                icerik = l.find_all("a")[1].text
                tarih = l.span.text[1:-1]
                link = adres + l.a.get("href")
                pg = requests.get(link)
                psp = BeautifulSoup(pg.content, "html.parser")
                content = ""
                for i in list(psp.find("div",{"class","tag-content"}).find_all("p")):
                    content += i.text.replace("\xa0"," ")

                array.append([title,content,tarih,bank])
            except:
                print(link)
                continue


def finans_gundem(array,bank):
    for i in tqdm(range(1,75)):
        page = requests.get(web_links["finans_gundem"].format(bank=bank,i=i))
        sp = BeautifulSoup(page.content,"html.parser")
        liste = sp.find("div",{"class":"srch"}).find_all("li")
        for l in liste: 
            try:
                title = l.strong.text
                tarih = l.small.text
                tarih = tarih[:tarih.find(",")]
                content = ""
                pg = requests.get(l.a.get("href"))
                psp = BeautifulSoup(pg.content,"html.parser")
                for i in list(psp.find("div",{"class":"dtail"}).find_all("p")[:-2]):
                    content += i.text.replace("\xa0"," ")
                if not content:
                    continue
                array.append([title,content,tarih,bank])
            except Exception as e:
                print(l.a.get("href"),"  ",e)
                continue    



In [ ]:
column_names = ["title","content","date","bank"]

array = []
for bank in banks:
    borsa_gundem(array,bank)
    bloomberght(array,bank)    
    hurriyet(array,bank)    
    finans_gundem(array,bank)
    data = pd.DataFrame(data=array,columns=column_names)
    data.to_csv("banks.csv",index=False)